In [4]:
from pyscipopt import Model, Eventhdlr, SCIP_EVENTTYPE,SCIP_HEURTIMING
import pyscipopt
import sys
from tqdm import tqdm
import random
import pandas as pd

In [5]:
class FixedVarsAtNode(Eventhdlr):
    """PySCIPOpt Event handler to write fixed vars of each node to a text file."""

    def __init__(self):
        
        #define the path of file
        # self.output_var_info_file = "var_info/var_info.txt"
        
        #others
        self.orgin_sol_info = pd.DataFrame(columns=['Node', 'ObjVal'])

        self.count=0
        self.count_limit=-1

        self.batch_size = 5000  # 设置每个文件记录的节点数
        self.batch_number = 0  # 当前文件的批次号
        

    def eventexec(self, event):
        if self.count>self.count_limit:
            
            obj_value_no_heuristic = self.model.getObjVal() 
            info_add_df ={'Node': self.count, 'ObjVal': obj_value_no_heuristic}
            self.orgin_sol_info = self.orgin_sol_info.append(info_add_df,ignore_index=True)

            # 判断是否达到记录的节点数
            if self.count % self.batch_size == 0:
                # 写入文件
                self.write_2_file(f'marks_output/origin/marks_output_{self.batch_number}')
                self.batch_number += 1
                # 创建一个新的DataFrame
                self.orgin_sol_info = pd.DataFrame(columns=['Node', 'ObjVal'])

            self.count += 1
        else:
            pass

         
      
    def write_2_file(self , filename):
        #输出为csv文件
        file_name = filename +"_y_info.csv"
        self.orgin_sol_info.to_csv(file_name, index=False)
        

    def eventinit(self):
        self.model.catchEvent(SCIP_EVENTTYPE.NODESOLVED, self)

    def eventexit(self):
        self.model.dropEvent(SCIP_EVENTTYPE.NODESOLVED, self)


In [ ]:
if __name__ == "__main__":
    # Specify the output file path
    #output_var_info_file = "var_info.txt"
    
    test = pyscipopt.scip.Model()
    test.setIntParam("heuristics/rins/freq",-1)
    test.readProblem("/Users/oukeikou/Desktop/sunruoyao/DATA/easy-sample/markshare_4_0.mps")
    print("read done")


    # Create and add event handler with the specified output file
    eventhdlr = FixedVarsAtNode()
    test.includeEventhdlr(eventhdlr, "FixedVarsAtNode", "Python event handler to write fixed variables after each solved node")

    # Optimize the problem
    test.optimize()
    print("optimized")
    
    # eventhdlr.write_2_file('markshare_4_0')

read done
original problem has 34 variables (30 bin, 0 int, 0 impl, 4 cont) and 4 constraints
presolving:
(round 1, fast)       4 del vars, 0 del conss, 4 add conss, 4 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       4 del vars, 4 del conss, 4 add conss, 4 chg bounds, 4 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver found nothing
(round 3, exhaustive) 4 del vars, 4 del conss, 4 add conss, 4 chg bounds, 4 chg sides, 0 chg coeffs, 4 upgd conss, 0 impls, 0 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int -, cont +), (fixed: bin -, int +, cont -)
   (0.0s) no symmetry present
presolving (4 rounds: 4 fast, 2 medium, 2 exhaustive):
 4 deleted vars, 4 deleted constraints, 4 added constraints, 4 tightened bounds, 0 added holes, 4 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 3